In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "")

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

In [4]:
from langchain.document_loaders import WikipediaLoader

raw_documents = WikipediaLoader(query="Lakshadweep").load()

In [14]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [6]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI (
    openai_api_version="2024-05-01-preview",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment="gpt-35-turbo-16k",
    temperature=0.0,
)

In [27]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [14]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
)

In [16]:
chain.invoke({
    'query': "Where is Lakshadweep located?",
})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Territory {id: "Lakshadweep"})-[:LOCATED_IN]->(c:Country)
RETURN c.id
Full Context:
[{'c.id': 'India'}]

> Finished chain.


{'query': 'Where is Lakshadweep located?',
 'result': 'Lakshadweep is located in India.'}